In [1]:
import numpy as np
import pandas as pd
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import pairwise2
from Bio.pairwise2 import format_alignment 
from Bio.Align import substitution_matrices
import math
import sys
import copy
import pprint

/home/janrose/.local/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
def Entropy_Calculation(microstates):
    Kb = 1.380649*10*np.exp(-23)
    return Kb*microstates

In [3]:
def SFree_E(Enthalpy,Entropy):
    T = 310
    return Enthalpy - T*Entropy

In [14]:
def calculate_directions(n,last_cord,Past_cords):
    directions = []
    k = [1,-1]
    for i in range(0,len(k)):
        result = last_cord[2] + k[i]
        if result < 0 or result > n:
            continue
        newCord = [last_cord[0],last_cord[1],result]
        exists = False
        for i in range(0,len(Past_cords)):
            if newCord == Past_cords[i]:
                exists = True
        if exists != True:
            directions.append(newCord)
            
    for i in range(0,len(k)):
        result = last_cord[1] + k[i]
        if result < 0 or result > n:
            continue
        newCord = [last_cord[0],result,last_cord[2]]
        exists = False
        for i in range(0,len(Past_cords)):
            if newCord == Past_cords[i]:
                exists = True
        if exists != True:
            directions.append(newCord)
            
    for i in range(0,len(k)):
        result = last_cord[0] + k[i]
        if result < 0 or result > n:
            continue
        newCord = [result,last_cord[1],last_cord[2]]
        exists = False
        for i in range(0,len(Past_cords)):
            if newCord == Past_cords[i]:
                exists = True
        if exists != True:
            directions.append(newCord)
            
    return directions  

In [15]:
def possible_connections(n,last_cord):
    directions = []
    k = [1,-1]
    for i in range(0,len(k)):
        result = last_cord[2] + k[i]
        if result < 0 or result > n:
            continue
        newCord = [last_cord[0],last_cord[1],result]
        directions.append(newCord)
    
    for i in range(0,len(k)):
        result = last_cord[1] + k[i]
        if result < 0 or result > n:
            continue
        newCord = [last_cord[0],result,last_cord[2]]
        directions.append(newCord)
    
    for i in range(0,len(k)):
        result = last_cord[0] + k[i]
        if result < 0 or result > n:
            continue
        newCord = [result,last_cord[1],last_cord[2]]
        directions.append(newCord)
    
    return directions

In [16]:
def backtracking(AA,Past_cords):
    totalHH = 0
    for i in range(0,len(Past_cords)):
        if AA[i] == 'H':
            locations = possible_connections(len(AA),Past_cords[i])
            for x in range(0,len(locations)):
                index = 0
                if locations[x] in Past_cords:
                    index = Past_cords.index(locations[x])
                    if (index != i + 1 or index != i - 1) and AA[index] == 'H':
                        totalHH += 1
                else:
                    continue
    return totalHH

In [17]:
def findfactors(num):
    num = num - 2
    factors = []
    for i in range(1,num+1):
        if num % i==0:
            factors.append(i)
    divisor = 0
    for i in factors:
        if i != num and i > divisor:
            divisor = i
    return divisor

In [18]:

def divideString(string, n):
    str_size = len(string) - 2
   
 
    # Check if string can be divided in n equal parts
    if n < 3:
        print ("Chunks too small to perform proper folding")
        return
    elif str_size % n != 0:
        print ("Invalid Input: String size is not divisible by n")
        return
    
 
    # Calculate the size of parts to find the division points
    part_size = n
    chunks = []
    allchunks = []
    k = 1
    for i in range(2,len(string)):
        if k != part_size:
            chunks.append(string[i])
            k += 1
        else: 
            chunks.append(string[i])
            
            allchunks.append(chunks)
            chunks = []
            k = 1
    
    return allchunks

In [20]:
def chunkFold(AA,index,chunks,last_cord,Past_cords,Maxcon,optimal_route):
    
    #We calculated the possible directions now we start placing AA 
    #then backtrack to make sure we have the best route
    if index == len(chunks):
        Connections = backtracking(AA,Past_cords)
        if Maxcon < Connections:
            Maxcon = Connections
            optimal_route = copy.deepcopy(Past_cords)
        Past_cords.pop()
        return Past_cords,Maxcon,optimal_route
        
        
    
    directions = calculate_directions(len(AA),last_cord,Past_cords) #I'm essentially hoping for each recursive call to store different directions
    
    
    #Here we're going backwards on the list. Last direction added is our final coordinate for the time being
    for i in range(len(directions) - 1 ,- 1,-1):
        Past_cords.append(directions[i])
        index += 1
        Past_cords, Maxcon,optimal_route = chunkFold(AA,index,chunks,directions[i],Past_cords,Maxcon,optimal_route)
        index = index - 1
        directions.pop()
    
    if bool(directions) == False:
        Past_cords.pop() #Need it or index messes up
        return Past_cords,Maxcon,optimal_route

In [21]:
def proteinfolding(AA,n):
    optimal_route = []
    Maxcon = -1 
    Lattice = [[[0 for k in range(n)] for j in range(n)] for i in range(n)]
    #Make all this global
    
    
    #Divide the AA string into chunks as shown in this paper https://www.wseas.org/multimedia/journals/circuits/2018/a225901-397.pdf
    chunks = divideString(AA,findfactors(n))
    
    
    ##Have to find a consistent way to start in the center
    center = math.ceil((n-1)/2)
    #print(math.ceil((n-2)/2))
    
    #Set the first two Letters at the center
    Lattice [center][center][center] = AA[0]
    Lattice [center][center][math.ceil((n-2)/2)] = AA[1]
    
    #Keep running list of points that have been used
    Past_cords = [[center,center,center],[center,center,math.ceil((n-2)/2)]]
    
    #Repeat this step for the number of chunks there are
    for i in range(0,len(chunks)):
        #print(chunks[i])
        #Past_cords[len(Past_cords) - 1]
        Past_cords,Maxcon,optimal_route = chunkFold(AA,0,chunks[i],Past_cords[len(Past_cords) - 1],Past_cords,Maxcon,optimal_route)
        Past_cords = copy.deepcopy(optimal_route)
        
    return optimal_route

In [22]:
def clean_lattice(size,lattice):
    #For now only focusing on cleaning rows
    count = 0
    for i in range(0,size):
        if sum(map(sum, lattice[count])) == 0:
            del lattice[count]
        else:
            count += 1
    #pprint.pprint(lattice)
    return lattice

In [26]:
def MostP(size,lattice):
    
    countP = 0
    maxP = 0
    face = 0
    #First we'll be checking for the face on top
    for x in range(0,size): #Checks row
        for y in range(0,size): #Checks element
            if lattice[0][x][y] == 1:
                 countP += 1
    if maxP < countP:
        maxP = countP
        face = 0
    
    countP = 0
    #We check the bottom face
    for x in range(0,size): #Checks row
        for y in range(0,size): #Checks element
             if lattice[len(lattice) - 1][x][y] == 1:
                countP += 1
    if maxP < countP:
        maxP = countP
        face = size - 1
        
    return maxP,face

In [28]:
#Main
#Testing Protein Folding

#Folding for the first peptide sequence
AA1 = "HPPHPHPH" #Peptide 1 #Can change
Conversion1 = [] 
for i in range(0,len(AA1)):
    if AA1[i] == "H":
        Conversion1.append(2)
    else: 
        Conversion1.append(1)
n1 = len(AA1)
LatticeSize = n1**3
Lattice = [[[0 for k in range(n1)] for j in range(n1)] for i in range(n1)]
optimal_route1 = proteinfolding(AA1,n1)
for i in range(0,len(optimal_route1)):
    Lattice[optimal_route1[i][0]][optimal_route1[i][1]][optimal_route1[i][2]] = Conversion1[i]
Lattice = clean_lattice(n1,Lattice)
print("This is the protein")
pprint.pprint(Lattice)
MaxPLattice,Ideal_face = MostP(n1,Lattice)

#Folding for the second peptide sequence
AA2 = "HPPHPHPH" #Can Change
Conversion2 = [] 
for i in range(0,len(AA2)):
    if AA2[i] == "H":
        Conversion2.append(2)
    else: 
        Conversion2.append(1)
n2 = len(AA2)
LatticeSize = n2**3
Peptide = [[[0 for k in range(n2)] for j in range(n2)] for i in range(n2)]
optimal_route2 = proteinfolding(AA2,n2)
for i in range(0,len(optimal_route2)):
    Peptide[optimal_route2[i][0]][optimal_route2[i][1]][optimal_route2[i][2]] = Conversion2[i]
Peptide = clean_lattice(n2,Peptide)
print("This is the peptide")
pprint.pprint(Peptide)
MaxPPeptide,Ideal_face = MostP(n2,Peptide)

#The correct way to do this would be to calculate entropy of one and the other I think
FEntropy = Entropy_Calculation(abs(n1*n1))
BEntropy = Entropy_Calculation(abs(n1*n1 - abs(MaxPPeptide - MaxPLattice)))
DEntropy = FEntropy - BEntropy 


#For now ignoring Enthalpy changes. 
StandardG = 0
if MaxPPeptide <= MaxPLattice:
    StandardG = SFree_E(MaxPPeptide,DEntropy)
else:
    StandardG = SFree_E(MaxPLattice,DEntropy)

print(StandardG)



This is the protein
[[[0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 2, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 0, 0],
  [0, 0, 0, 1, 2, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 2, 0, 0, 0],
  [0, 0, 0, 1, 2, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0]]]
This is the peptide
[[[0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 2, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0